Berikut adalah dokumentasi dalam format Markdown untuk proyek klasifikasi sampah menggunakan ResNet, diikuti oleh implementasi kode.

---

# Klasifikasi Sampah Menggunakan ResNet dengan TensorFlow

## Pendahuluan
Klasifikasi sampah menggunakan deep learning adalah metode yang dapat membantu otomatisasi dalam pemilahan dan pengelolaan limbah secara efisien. Model berbasis arsitektur Residual Network (ResNet) mampu mengenali jenis sampah dengan menggunakan dataset gambar yang dilabeli, yang berpotensi memperbaiki proses daur ulang otomatis. ResNet sering digunakan dalam pengenalan gambar karena kemampuannya mengatasi masalah vanishing gradient melalui blok residual yang mempertahankan informasi penting dari lapisan sebelumnya [(He et al., 2016)](https://consensus.app/papers/resnet-deep-learning-he/9c586abb395db4c2a24d1a4e18388f23/).

## Lingkungan dan Pengaturan Awal
- **Library yang Digunakan**: TensorFlow digunakan sebagai framework utama untuk deep learning, bersama pustaka `pandas`, `numpy`, dan `matplotlib` untuk manipulasi data dan visualisasi. `wandb` digunakan untuk pelacakan metrik model dan logging eksperimen, yang membantu dalam memantau performa model dan stabilitas selama pelatihan [(Biewald, 2020)](https://consensus.app/papers/weights-biases-open-source-biewald/e1289d2819c3a3d8cb70d44f6a5587b5/).
- **Dataset TrashNet**: Dataset gambar dari berbagai jenis sampah, seperti kertas, plastik, dan kaca, digunakan untuk melatih model. TrashNet menyediakan data yang beragam dan relevan untuk kebutuhan klasifikasi sampah otomatis [(Yang & Thung, 2017)](https://consensus.app/papers/classification-trash-using-deep-learning-yang-thung/e65627e5d9095e9232ff2f193fef10a7/).

## Arsitektur Model ResNet
ResNet menggunakan struktur blok residual untuk memungkinkan sinyal informasi melewati jaringan dengan minimal gangguan, mengatasi masalah vanishing gradient yang umum pada jaringan dalam [(He et al., 2016)](https://consensus.app/papers/resnet-deep-learning-he/9c586abb395db4c2a24d1a4e18388f23/).

- **Blok Residual**: Blok residual memungkinkan shortcut connections yang membawa input asli dari satu lapisan ke lapisan berikutnya, menjaga integritas informasi dari lapisan awal [(Szegedy et al., 2015)](https://consensus.app/papers/going-deeper-convolutions-szegedy/1a4e1e4b0f45310d711eaac8ae945a17/).
- **Arsitektur Model**: Model dimulai dengan lapisan konvolusi besar, diikuti dengan pooling dan beberapa blok residual. Global Average Pooling digunakan di akhir jaringan, yang memberikan stabilitas saat menangani data gambar dalam jumlah besar dan bervariasi.

## Pembagian Data
Untuk melatih model dengan data yang seimbang, dataset dibagi menjadi tiga set: pelatihan, validasi, dan pengujian. Pembagian manual memastikan data terdistribusi secara merata, meminimalkan risiko bias [(James et al., 2013)](https://consensus.app/papers/introduction-statistical-learning-james-et-al/09d17878513a84a3e95c748c8b6e5e93/).

## Preprocessing dan Augmentasi Data
- **Normalisasi dan Deteksi Blur**: Gambar normalisasi dan deteksi blur diterapkan pada setiap gambar agar kualitas gambar menjadi lebih baik sebelum dimasukkan ke model. Normalisasi dapat meningkatkan akurasi dan stabilitas model [(Krizhevsky et al., 2012)](https://consensus.app/papers/image-net-classification-deep-convolutional-krizhevsky-sutskever-hinton/2e5c8ac2f47d97cf9fc9b515f9aa2b24/).
- **Augmentasi Data**: Augmentasi seperti flip horizontal, rotasi, dan penyesuaian kecerahan digunakan untuk memperkaya data gambar. Teknik ini terbukti meningkatkan kemampuan generalisasi model pada data yang belum pernah dilihat sebelumnya [(Shorten & Khoshgoftaar, 2019)](https://consensus.app/papers/survey-image-data-augmentation-deep-shorten-khoshgoftaar/c2c52f2dd34891f128cc27ff18bc9e48/).

## Pelatihan Model
Model dilatih pada data pelatihan dan dievaluasi pada data validasi untuk menguji akurasi dan stabilitas. Hanya model yang mencapai akurasi dan stabilitas tertentu yang disimpan, untuk menghindari model overfitting atau underfitting [(Goodfellow et al., 2016)](https://consensus.app/papers/deep-learning-goodfellow-bengio-courville/90a54b117d70e0309978f7d0af87a4ef/).

## Evaluasi dan Prediksi Model
Evaluasi model menggunakan data pengujian untuk menghitung loss dan akurasi, memberikan gambaran yang lebih baik tentang performa model pada data baru. Akurasi tinggi menunjukkan kemampuan model dalam mengklasifikasikan sampah dengan baik [(Russakovsky et al., 2015)](https://consensus.app/papers/imagenet-large-scale-visual-recognition-challenge-russakovsky-deng/66e0a771282f27a07b05f98f315d2be3/).

---

In [3]:
import tensorflow as tf # type: ignore
import numpy as np
import random
from PIL import Image
from tensorflow.keras.preprocessing import image  # type: ignore
from tensorflow.keras.applications import ResNet50  # type: ignore
from tensorflow.keras import layers, models  # type: ignore
import wandb # type: ignore
import tensorflow as tf # type: ignore
import sys
import numpy as np
import random
import wandb
import os
from tensorflow.keras.optimizers import Adam # type: ignore
from tensorflow.keras.callbacks import ModelCheckpoint # type: ignore


# Memeriksa dan mengatur GPU
gpus = tf.config.list_physical_devices('GPU')
if not gpus:
    print("Tidak ada GPU yang terdeteksi. Program ini memerlukan GPU untuk dijalankan.")
    sys.exit(1)  # Menghentikan program dengan kode status 1

try:
    tf.config.experimental.set_memory_growth(gpus[0], True)
    print("Pengaturan memori GPU berhasil.")
except Exception as e:
    print(f"Kesalahan saat mengatur memori GPU: {e}")
    sys.exit(1)  # Menghentikan program jika terjadi kesalahan saat mengatur GPU


2024-11-08 12:37:28.718049: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-08 12:37:28.718180: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-08 12:37:28.719596: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-08 12:37:28.728147: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-08 12:37:29.995541: W tensorflow/compiler/tf2

Pengaturan memori GPU berhasil.


2024-11-08 12:37:31.654415: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-08 12:37:31.700204: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-08 12:37:31.700480: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [9]:
# Fungsi untuk memproses dan melakukan augmentasi pada gambar
import os
import random
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from datasets import load_dataset # type: ignore
import wandb

def preprocess_and_augment_images(dataset, num_images=0, target_size=(224, 224)):
    train_images = []
    train_labels = []

    for i in range(min(num_images, len(dataset['train']))):
        try:
            img = dataset['train'][i]['image'].convert('RGB').resize(target_size)
            label = dataset['train'][i]['label']
            img_array = np.array(img) / 255.0
            
            # Deteksi blur dan augmentasi gambar
            if np.std(img_array) < 0.05:
                img_array *= 2  # Tingkatkan kontras
            
            augment_type = random.choice(['flip', 'rotate', 'brightness'])
            if augment_type == 'flip':
                img_array = np.fliplr(img_array)
            elif augment_type == 'rotate':
                img_array = np.rot90(img_array)
            elif augment_type == 'brightness':
                factor = random.uniform(0.5, 1.5)
                img_array = np.clip(img_array * factor, 0, 1)

            train_images.append(img_array)
            train_labels.append(label)

        except Exception as e:
            print(f"Error processing image {i + 1}: {e}")

    return np.array(train_images), np.array(train_labels)

# Fungsi untuk membuat model ResNet50
def create_resnet50(input_shape, num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model .output
    x = GlobalAveragePooling2D()(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=outputs)

    for layer in base_model.layers:
        layer.trainable = False  # Membekukan lapisan dasar

    return model

# Fungsi untuk melatih dan menyimpan model dengan kondisi tertentu
def train_and_save_model(train_images, train_labels, val_images, val_labels, num_classes, epochs=10):
    # Membuat model menggunakan ResNet50
    model = create_resnet50(input_shape=(224, 224, 3), num_classes=num_classes)

    # Mengompilasi model dengan optimizer dan loss function
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Melatih model dan menyimpan history
    history = model.fit(train_images, train_labels, validation_data=(val_images, val_labels), epochs=epochs, batch_size=64)

    if history.history['val_accuracy'][-1] >= 0.80 and history.history['accuracy'][-1] >= 0.80:
        # Log ke wandb
        wandb.log({
            "train_accuracy": history.history['accuracy'][-1],
            "val_accuracy": history.history['val_accuracy'][-1],
            "train_loss": history.history['loss'][-1],
            "val_loss": history.history['val_loss'][-1]
        })
        model.save('simpan_resnet50_model_best.h5')
    # Memeriksa akurasi pada epoch terakhir
   
    model.save('simpan_resnet50_model.h5')  # Simpan model jika akurasi validasi >= 80%
    
    return model

# Fungsi untuk mengevaluasi model pada data uji
def evaluate_model(model, test_images, test_labels):
    loss, accuracy = model.evaluate(test_images, test_labels)
    print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

# Fungsi untuk prediksi gambar
def predict_image(model, img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = np.array(img) / 255.0
    pred = model.predict(np.expand_dims(img_array, axis=0))
    predicted_class = np.argmax(pred, axis=-1)
    return predicted_class

# Fungsi utama untuk menjalankan pelatihan dan evaluasi
def main():
    # Inisialisasi wandb dan login menggunakan API Key
    wandb.login(key='967fafd5558eeeff3ce5681cf55c71633438428d')  # Ganti dengan API Key kamu
   
    wandb.init(project="trash-classification")
    # Load dataset TrashNet dari Hugging Face
    dataset = load_dataset("garythung/trashnet")

    # Preprocessing dan augmentasi data
    train_images, train_labels = preprocess_and_augment_images(dataset, num_images=1000)
    val_images, val_labels = preprocess_and_augment_images(dataset, num_images=200)
    
    # Melatih dan menyimpan model
    modelVal = train_and_save_model(train_images, train_labels, val_images, val_labels, num_classes=6, epochs=400)
   
    # Evaluasi model pada data uji
    test_images, test_labels = preprocess_and_augment_images(dataset, num_images=100)
    evaluate_model(modelVal, test_images, test_labels)

    # Prediksi gambar
    predicted_class = predict_image(modelVal, "/workspaces/aitask/notebooks/image/sampah.jpg")
    print(predicted_class)

    # Logout wandb setelah eksperimen selesai
    wandb.finish()

# Menjalankan fungsi utama
if __name__ == "__main__":
    main()

2024-11-08 15:29:17.506244: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-08 15:29:17.506334: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-08 15:29:17.508058: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-08 15:29:17.519475: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-08 15:29:19.175188: W tensorflow/compiler/tf2

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

2024-11-08 15:37:49.710516: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-08 15:37:49.967074: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-08 15:37:49.967497: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

94765736/94765736 [==============================] - 1s 0us/step
Epoch 1/300


...

 1/32 [..............................] - ETA: 6:24 - loss: 1.8943 - accuracy: 0.3750

I0000 00:00:1731080302.541569     141 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


32/32 [==============================] - 20s 243ms/step - loss: 1.0958 - accuracy: 0.4480 - val_loss: 0.8640 - val_accuracy: 0.2750
Epoch 2/300
32/32 [==============================] - 3s 96ms/step - loss: 0.9542 - accuracy: 0.4910 - val_loss: 0.9825 - val_accuracy: 0.0000e+00
Epoch 3/300
32/32 [==============================] - 3s 100ms/step - loss: 0.9389 - accuracy: 0.5060 - val_loss: 0.7862 - val_accuracy: 0.8400
Epoch 4/300
32/32 [==============================] - 3s 95ms/step - loss: 0.9234 - accuracy: 0.4930 - val_loss: 0.8096 - val_accuracy: 0.5100
Epoch 5/300
32/32 [==============================] - 3s 95ms/step - loss: 0.9134 - accuracy: 0.5240 - val_loss: 0.8733 - val_accuracy: 0.1100
Epoch 6/300
32/32 [==============================] - 3s 105ms/step - loss: 0.8971 - accuracy: 0.5690 - val_loss: 0.8306 - val_accuracy: 0.3850
Epoch 7/300
32/32 [==============================] - 3s 97ms/step - loss: 0.8869 - accuracy: 0.6110 - val_loss: 0.8026 - val_accuracy: 0.4900
Epoch 8/30

wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


train_accuracy,▁
train_loss,▁
val_accuracy,▁
val_loss,▁
train_accuracy,0.774
train_loss,0.61548
val_accuracy,0.775
val_loss,0.63592


Jika Hadi